In [6]:
#import necessary libraries
import cv2
import csv 
import mediapipe as mp
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
model = load_model(r'H:\Sajal Video\Models\model.tf') #load model that was saved in training session

#mediapipe settings
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# field names initialization
right_wrist_x = right_elbow_x = right_shoulder_x = left_wrist_x = left_elbow_x = left_shoulder_x = right_hip_x = right_knee_x = right_ankle_x = left_hip_x = left_knee_x = left_ankle_x = -1
right_wrist_y = right_elbow_y =  right_shoulder_y =  left_wrist_y =  left_elbow_y =  left_shoulder_y = right_hip_y =  right_knee_y = right_ankle_y = left_hip_y = left_knee_y = left_ankle_y = -1

# get the video, set camera id for live camera use case, for example set 0 if you have one camera, i.e. web cam
cap = cv2.VideoCapture(r'H:\Sajal Video\shoplifting\Pocket 3rd Part.mp4')
img_num = 1 #for gathering window_len amount image frame
window_len = 32 # set window length as per your traininng
rows = [] # list where window_len amount data are stored
color = (0,0,255) # color for normal customer
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read() #read the frame
        if image is None: #if it is the last frame, break the loop
            break
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.,
            continue

        # To improve performance, optionally mark the image as not writeable to,
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image_new = image.copy()
        image_height, image_width, _ = image_new.shape
        if not results.pose_landmarks: #if the image doesn't contain a person, i.e. no landmark is detected then ignore
            continue
        #get the all necessary landmark coordinates
        right_wrist_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * image_width
        right_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * image_height

        right_elbow_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].x * image_width
        right_elbow_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].y * image_height

        right_shoulder_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width
        right_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_height

        left_wrist_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * image_width
        left_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * image_height

        left_elbow_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x * image_width
        left_elbow_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y * image_height

        left_shoulder_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width
        left_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_height

        right_hip_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x * image_width
        right_hip_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y * image_height

        right_knee_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x * image_width
        right_knee_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y * image_height

        right_ankle_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x * image_width
        right_ankle_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y * image_height

        left_hip_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x * image_width
        left_hip_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y * image_height

        left_knee_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x * image_width
        left_knee_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y * image_height

        left_ankle_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x * image_width
        left_ankle_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y * image_height
        
        
        #stack them serially as you have collected the datapoints
        row = [
            left_shoulder_x, left_shoulder_y,
            right_shoulder_x, right_shoulder_y,
            left_elbow_x, left_elbow_y,
            right_elbow_x, right_elbow_y,
            left_wrist_x, left_wrist_y ,
            right_wrist_x, right_wrist_y,
            left_hip_x, left_hip_y,
            right_hip_x, right_hip_y,
            left_knee_x, left_knee_y,
            right_knee_x,right_knee_y,
            left_ankle_x, left_ankle_y,
            right_ankle_x, right_ankle_y,
            
            
        ]
        """
        row = [
            right_wrist_x, right_wrist_y,
            right_elbow_x, right_elbow_y,
            right_shoulder_x, right_shoulder_y,
            left_wrist_x, left_wrist_y ,
            left_elbow_x, left_elbow_y,
            left_shoulder_x, left_shoulder_y,
            right_hip_x, right_hip_y,
            right_knee_x,right_knee_y,
            right_ankle_x, right_ankle_y,
            left_hip_x, left_hip_y,
            left_knee_x, left_knee_y,
            left_ankle_x, left_ankle_y
        ]
        """
        rows.append(row)
        cv2.rectangle(image, (int(right_shoulder_x),int(right_shoulder_y)), (int(left_ankle_x), int(left_ankle_y)), color, 2) #create a bounding box around the customer
        if np.array(rows).shape[0] % window_len == 0: #if window length amount data is collected
            rows_2 = rows            rows = [rows] # for formating purpose
            X_test = np.array(rows) #converting into numpy array for feeding it into the model properly
            del rows_2[0]# discarding the first item of the list for next window
            rows = rows_2            out = model.predict(X_test) #get the prediction
            print(np.argmax(out, axis=1)[0]) 
            if np.argmax(out, axis=1)[0] == 1:#if the prediction is 1, change the color to Red
                color = (255,0,0)
            if np.argmax(out, axis=1)[0] == 0: #if it is 0, change back the color to green
                color = (0,0,255)
        img_num = img_num + 1 #count the number of frames passed

        # Draw the pose annotation on the image.,
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # Flip the image horizontally for a selfie-view display.,
        cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27: #break when escape button is pressed
            break
cap.release()
cv2.destroyAllWindows()

0
1
0
1
1
1
1
0
1
1
1
1
